In [8]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from Pretraitement import Pretraitements
base_path = "C:\\Users\\ACER\\Desktop\\UTC\\SY09\\PROJET\\sy09_russian-troll-tweets"

## Regroupement des lignes par auteur avec fusion de la colonne content

In [2]:
p = 1
original_data=pd.read_csv(os.path.join(base_path, "Data\combined_csv.csv"), skiprows=lambda i: i>0 and random.random() > p)
original_data=original_data[original_data['language']=='English']

In [ ]:
original_data["content"] = original_data["content"].astype(str)
data_by_account = original_data.groupby(["author", "account_category"])["content"].apply(" ".join).reset_index()


In [ ]:
data_by_account.head()

## Calcul des scores

In [ ]:
pt=Pretraitements(data_by_account)
pt.script_score()

In [15]:
data_by_account.to_csv(os.path.join(base_path, "Data\data_with_scores\data_by_account_with_scores.csv"))

## Analyse exploratoire

In [20]:
account_data=pd.read_csv(os.path.join(base_path, "Data\data_with_scores\data_by_account_with_scores.csv"))
selected_cat = ['HashtagGamer', 'LeftTroll', 'NewsFeed', 'RightTroll']
dataset = account_data.loc[account_data["account_category"].isin(selected_cat)]

In [21]:
dataset

,Unnamed: 0,author,account_category,content,score_word_HashtagGamer,score_word_LeftTroll,score_word_NewsFeed,score_word_RightTroll,score_hashtag_HashtagGamer,score_hashtag_LeftTroll,score_hashtag_NewsFeed,score_hashtag_RightTroll
0,0,10_GOP,RightTroll,"""We have a sitting Democrat US Senator on tria...",7.888854,10.576907,7.373520,17.507590,0.000000,0.000000,0.000000,0.000000
2,2,1ERIK_LEE,RightTroll,Why is someone even against the #petition? I'l...,0.027631,0.031460,0.010645,0.027038,0.000000,0.011002,0.000000,0.000000
4,4,2NDHALFONION,RightTroll,'@HalfOnionInABag Follow the other half an oni...,0.045599,0.042946,0.000000,0.062305,0.000000,0.000000,0.000000,0.000000
6,6,4EVER_SUSAN,RightTroll,#Raiders defense playing hungry .. Bending and...,0.735350,0.904342,0.298889,0.922559,0.000000,0.000000,0.000000,0.014282
7,7,4MYSQUAD,LeftTroll,#blacklivesmatter #blm #equality #equalrights ...,38.599808,58.989807,39.244718,39.832264,0.000000,16.672661,0.799389,0.380156
...,...,...,...,...,...,...,...,...,...,...,...,...
2153,2153,_NICKLUNA_,RightTroll,Send ur (least) fave #RINOs a msg... Send them...,53.427207,72.144976,49.408046,113.075239,0.009314,0.585849,4.035897,2.428882
2154,2154,_RONBEN,RightTroll,Which homosexual sex act do you disapprove of ...,0.561859,0.636060,0.337416,0.588206,0.000000,0.004194,0.000000,0.012263
2155,2155,_RUBY_WILLS_,RightTroll,Liars @CarlyFiorina and #GOP frozen in denial ...,3.188472,3.756781,1.889601,4.532060,0.000000,0.048864,0.000000,0.445384
2157,2157,_SHERYLGILBERT,RightTroll,I've never had a hobby. That is so weird! And ...,0.982081,0.745730,0.189025,0.587134,0.000000,0.024590,0.000000,0.000000


In [67]:
score_max = dataset[["score_word_HashtagGamer", "score_word_LeftTroll", "score_word_NewsFeed", "score_word_RightTroll"]].idxmax(axis=1).apply(lambda x: x.split("_")[2])

In [68]:
accuracy_score(dataset["account_category"], score_max)

0.7900874635568513

## Classification KNN

In [76]:
X_variables = ["score_word_HashtagGamer", "score_hashtag_HashtagGamer", "score_word_LeftTroll", "score_hashtag_LeftTroll", "score_word_NewsFeed", "score_hashtag_NewsFeed", "score_word_RightTroll", "score_hashtag_RightTroll"]
y_variables = ["account_category"]
X_dataset = dataset[X_variables]
y_dataset = dataset[y_variables]
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, test_size=0.33, random_state=0)

### Avec PCA

In [77]:
pca = PCA(n_components=2)
pca.fit(X_train)
print(pca.explained_variance_ratio_)
p_compo = pca.transform(X_train)

[0.69988058 0.26722392]


In [78]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(p_compo, y_train)
X_test_pca = pca.transform(X_test)
y_pred = neigh.predict(X_test_pca)
accuracy_score(y_test, y_pred)

C:\Users\ACER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.6411764705882353

### Sans PCA

In [79]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\ACER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.8205882352941176

### Sans PCA avec validation croisée fait maison

In [82]:
accuracies = np.empty(100)
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, test_size=0.33, random_state=i)
    neigh = KNeighborsClassifier(n_neighbors=5)
    neigh.fit(X_train, y_train)
    y_pred = neigh.predict(X_test)
    accuracies[i] = accuracy_score(y_test, y_pred)


In [81]:
pd.DataFrame(accuracies).describe()

,0
count,100.000000
mean,0.825559
std,0.016845
min,0.785294
25%,0.814706
50%,0.823529
75%,0.838235
max,0.873529
